In [1]:
import sys
import pandas as pd
import numpy as np
import matplotlib as mpl
import matplotlib.dates as dates
import matplotlib.pyplot as plt
import datetime
import itertools
import copy
sys.path.append("C:/dev/pyktrader3/")
from pycmqlib3.utility import misc
from bktest import backtest
import pycmqlib3.analytics.data_handler as dh

In [2]:
ferrous_products_mkts = ['rb', 'hc', 'i', 'j', 'jm']
ferrous_mixed_mkts = ['ru', 'FG', 'ZC', 'SM', "SF"]
base_metal_mkts = ['cu', 'al', 'zn', 'pb', 'ni', 'sn']
precious_metal_mkts = ['au', 'ag']
ind_metal_mkts = ferrous_products_mkts + ferrous_mixed_mkts + base_metal_mkts  
petro_chem_mkts = ['l', 'pp', 'v', 'TA', 'MA', 'bu', 'sc', 'fu', 'eg']
ind_all_mkts = ind_metal_mkts + petro_chem_mkts
ags_oil_mkts = ['m', 'RM', 'y', 'p', 'OI', 'a', 'c', 'cs'] #, 'b']
ags_soft_mkts = ['CF', 'SR', 'jd', 'AP'] #, 'sp', 'CJ', 'UR']
ags_all_mkts = ags_oil_mkts + ags_soft_mkts
eq_fut_mkts = ['IF', 'IH', 'IC']
bond_fut_mkts = ['T', 'TF']
fin_all_mkts = eq_fut_mkts + bond_fut_mkts
commod_all_mkts = ind_all_mkts + ags_all_mkts + precious_metal_mkts
all_markets = commod_all_mkts + fin_all_mkts
daily_start_dict = { 'c': datetime.date(2011,1,1), 'm': datetime.date(2011,1,1),
    'y': datetime.date(2011,1,1), 'l': datetime.date(2011,1,1), 'rb':datetime.date(2011,1,1),
    'p': datetime.date(2011,1,1), 'cu':datetime.date(2011,1,1), 'al':datetime.date(2011,1,1),
    'zn':datetime.date(2011,1,1), 'au':datetime.date(2011,1,1), 'v': datetime.date(2011,1,1),
    'a': datetime.date(2011,1,1), 'ru':datetime.date(2011,1,1), 'ag':datetime.date(2012,6,1),
    'i': datetime.date(2014,1,1), 'j': datetime.date(2012,6,1), 'jm':datetime.date(2013,7,1),
    'CF':datetime.date(2012,5,1),  'TA':datetime.date(2012,4,15),
    'PM':datetime.date(2013,10,1), 'RM':datetime.date(2013,1,1),  'SR':datetime.date(2013,1,1),
    'FG':datetime.date(2013,1,1),  'OI':datetime.date(2013,5,1),  'RI':datetime.date(2013,1,1),
    'WH':datetime.date(2014,5,1),  'pp':datetime.date(2014,5,1),
    'IF':datetime.date(2010,5,1),  'MA':datetime.date(2012,1,1),  'TF':datetime.date(2014,6,1),
    'IH':datetime.date(2015,5,1),  'IC':datetime.date(2015,5,1),  'cs':datetime.date(2015,2,1),
    'jd':datetime.date(2014,5,1),  'ni':datetime.date(2015,5,1),  'sn':datetime.date(2017,4,1),
    'ZC':datetime.date(2013,11,1), 'hc':datetime.date(2016, 4, 1), 'SM': datetime.date(2017,1,1),
    'SF': datetime.date(2017,9,1), 'CY': datetime.date(2017, 9, 1), 'AP': datetime.date(2018, 1, 1),
    'TS': datetime.date(2018, 9, 1), 'fu': datetime.date(2018, 9, 1), 'sc': datetime.date(2018, 8, 1),
    'b': datetime.date(2018, 1, 1), 'pb': datetime.date(2016, 7, 1), 'bu': datetime.date(2015,9,15),
    'T':datetime.date(2015,6,1), 'ss': datetime.date(2020, 5, 1), 'sp': datetime.date(2019, 5, 1),
    'CJ': datetime.date(2019, 8, 9), 'UR': datetime.date(2019, 8, 9), 'SA': datetime.date(2020, 1, 1),
    'eb': datetime.date(2020, 1, 1), 'eg': datetime.date(2019, 5, 1), 'rr': datetime.date(2019, 9, 1),
    'pg': datetime.date(2020, 9, 1), 'lu': datetime.date(2020, 8, 1),}
min_start_dict = { 'c': datetime.date(2011,1,1), 'm': datetime.date(2011,1,1),
    'y': datetime.date(2011,1,1), 'l': datetime.date(2011,1,1), 'rb':datetime.date(2011,1,1),
    'p': datetime.date(2011,1,1), 'cu':datetime.date(2011,1,1), 'al':datetime.date(2011,1,1),
    'zn':datetime.date(2011,1,1), 'au':datetime.date(2011,1,1), 'v': datetime.date(2011,1,1),
    'a': datetime.date(2011,1,1), 'ru':datetime.date(2011,1,1), 'ag':datetime.date(2012,6,1),
    'i': datetime.date(2014,1,1), 'j': datetime.date(2012,6,1), 'jm':datetime.date(2013,7,1),
    'CF':datetime.date(2012,5,1),  'TA':datetime.date(2012,4,15),
    'PM':datetime.date(2013,10,1), 'RM':datetime.date(2013,1,1),  'SR':datetime.date(2013,1,1),
    'FG':datetime.date(2013,1,1),  'OI':datetime.date(2013,5,1),  'RI':datetime.date(2013,1,1),
    'WH':datetime.date(2014,5,1),  'pp':datetime.date(2014,5,1),
    'IF':datetime.date(2010,5,1),  'MA':datetime.date(2012,1,1),  'TF':datetime.date(2014,6,1),
    'IH':datetime.date(2015,5,1),  'IC':datetime.date(2015,5,1),  'cs':datetime.date(2015,2,1),
    'jd':datetime.date(2014,5,1),  'ni':datetime.date(2015,5,1),  'sn':datetime.date(2017,4,1),
    'ZC':datetime.date(2013,11,1), 'hc':datetime.date(2016, 4, 1), 'SM': datetime.date(2017,1,1),
    'SF': datetime.date(2017,9,1), 'CY': datetime.date(2017, 9, 1), 'AP': datetime.date(2018, 1, 1),
    'TS': datetime.date(2018, 9, 1), 'fu': datetime.date(2018, 9, 1), 'sc': datetime.date(2018, 8, 1),
    'b': datetime.date(2018, 1, 1), 'pb': datetime.date(2016, 7, 1), 'bu': datetime.date(2015,9,15),
    'T':datetime.date(2015,6,1), 'ss': datetime.date(2020, 5, 1), 'sp': datetime.date(2019, 5, 1),
    'CJ': datetime.date(2019, 8, 9), 'UR': datetime.date(2019, 8, 9), 'SA': datetime.date(2020, 1, 1),
    'eb': datetime.date(2020, 1, 1), 'eg': datetime.date(2019, 5, 1), 'rr': datetime.date(2019, 9, 1),
    'pg': datetime.date(2020, 9, 1), 'lu': datetime.date(2020, 8, 1),}

In [3]:
def minbar_agg(df, minlist = [1500, 1930], index_col = 'datetime', field_funcs = {}):
    from collections import OrderedDict
    if index_col != None:
        adf = df.reset_index().copy()
    else:
        adf = df.copy()
    adf['split_idx'] = 0
    func_dict = OrderedDict()
    for col_name in adf.columns:
        if type(col_name).__name__ in ['str', 'unicode']:
            col = str(col_name.strip())
        else:
            col = col_name
        if col in ['split_idx', 'instID', 'exch', 'contract', 'open', 'date', 'datetime']:
            func_dict[col] = 'first'
        elif (col in ['volume', 'lr', 'lr2', 'lr3', 'lr4', 'lr2_up']):
            func_dict[col] = 'sum'
        elif (col in ['close', 'openInterest', 'min_id', 'shift']):
            func_dict[col] = 'last'
        elif (col in ['high']):
            func_dict[col] = 'max'
        elif (col in ['low']):
            func_dict[col] = 'min'
        elif col in field_funcs:
            func_dict[col] = field_funcs[col]
    adf['min_idx'] = 0
    for idx, mid in enumerate(minlist):
        flag = (adf['min_id']>=mid) & ((adf['min_id'].shift(1) < mid) | (adf['date'].shift(1) < adf['date']))  
        adf.loc[flag, 'min_idx'] = 1
        adf.loc[flag, 'split_idx'] = idx
    adf['cum_idx'] = adf['min_idx'].cumsum()
    adf['split_idx'] = adf['split_idx'].fillna(method='ffill')
    xdf = adf.groupby([adf['cum_idx']]).aggregate(func_dict)
    for idx in ['min_idx', 'cum_idx']:
        if idx in xdf.columns:
            xdf.drop(idx, axis = 1, inplace=True)
    xdf = xdf.reset_index()
    if index_col != None:
        xdf = xdf.set_index(index_col)
    #xdf = xdf.sort_values(by=['date', 'min_id'])
    return xdf


In [4]:
field_list = ['open', 'high', 'low', 'close', 'volume', 'openInterest', 'contract', 'shift']
mfield_list = ['open', 'high', 'low', 'close', 'volume', 'openInterest', 'contract', 'shift', 'datetime']
commod_mkts = ['rb', 'hc', 'i', 'j', 'jm', 'ru', 'FG', 'ZC', 'cu', 'al', 'zn', 'pb', 'ni', 'sn', \
               'l', 'pp', 'v', 'TA', 'MA', 'bu', 'sc', 'm', 'RM', 'y', 'p', 'OI', 'a', 'c', 'cs', 'CF', 'SR', 'jd']

start_date = datetime.date(2015, 1, 1)
end_date = datetime.date(2021, 4, 7)
pnl_tenors = ['1y', '2y', '3y', '4y', '5y', '6y', '8y', '10y']

sim_products = ind_all_mkts + ags_all_mkts

need_shift = 1
freq = 'm'
delim = '-'
file_folder = "C:\\dev\\data\\data_cache\\"

if freq == 'd':
    sim_start_dict = daily_start_dict
else:
    sim_start_dict = min_start_dict

dbtbl_prefix = ''

args = {'roll_rule': '-30b', 'freq': freq, 'need_shift': need_shift, 'dbtbl_prefix': dbtbl_prefix}
ferrous_products_args = args
ferro_mixed_mkt_args = args
base_args = {'roll_rule': '-30b', 'freq': freq, 'need_shift': need_shift, 'dbtbl_prefix': dbtbl_prefix}
base2_args = {'roll_rule': '-40b', 'freq': freq, 'need_shift': need_shift, 'dbtbl_prefix': dbtbl_prefix}
v_args  = {'roll_rule': '-28b', 'freq': freq, 'need_shift': need_shift, 'dbtbl_prefix': dbtbl_prefix}
sc_args = {'roll_rule': '-20b', 'freq': freq, 'need_shift': need_shift, 'dbtbl_prefix': dbtbl_prefix}
eq_args = {'roll_rule': '-1b', 'freq': freq, 'need_shift': need_shift, 'dbtbl_prefix': dbtbl_prefix}
bond_args = {'roll_rule': '-30b', 'freq': freq, 'need_shift': need_shift, 'dbtbl_prefix': dbtbl_prefix}
precious_args = {'roll_rule': '-25b', 'freq': freq, 'need_shift': need_shift, 'dbtbl_prefix': dbtbl_prefix}


df_list = []
bar_split_setup = {"s2": {"split_dict": [300, 1500], 
                          "extra_fields": {"exec_price1": ['open', 'mean', [(305, 307), (1505, 1507)]],
                                           "exec_price2": ['open', 'mean', [(310, 312), (1510, 1512)]],
                                           "exec_price3": ['open', 'mean', [(315, 320), (1515, 1520)]],
                                           "exec_price4": ['open', 'mean', [(320, 325), (1520, 1525)]],
                                           "lr_mean": ['lr', 'mean', []],
                                           "lr_std": ['lr2', 'mean', []],
                                           "up_std": ['lr2_up', 'mean', []],
                                           "lr_skew": ['lr3', 'mean', []],
                                           "lr_kurt": ['lr4', 'mean', []],
                                           "min_cnt": ['lr', 'count', []],
                                           "lr_abs": ["lr_abs", 'mean', []],
                                           "vol_amt": ['vol_amt', 'sum', []],
                                           "vol_mf": ['signed_volamt', 'sum', []],
                                           "oi_mf": ['oi_amt', 'sum', []],
                                          }},
                   "s21": {"split_dict": [1500, 1930], 
                          "extra_fields": {"exec_price1": ['open', 'mean', [(1505, 1507), (1935, 1937)]],
                                           "exec_price2": ['open', 'mean', [(1510, 1512), (1940, 1942)]],
                                           "exec_price3": ['open', 'mean', [(1515, 1520), (1945, 1950)]],
                                           "exec_price4": ['open', 'mean', [(1520, 1525), (1950, 1955)]],
                                           "lr_mean": ['lr', 'mean', []],
                                           "lr_std": ['lr2', 'mean', []],
                                           "up_std": ['lr2_up', 'mean', []],
                                           "lr_skew": ['lr3', 'mean', []],
                                           "lr_kurt": ['lr4', 'mean', []],
                                           "min_cnt": ['lr', 'count', []],
                                           "lr_abs": ["lr_abs", 'mean', []],
                                           "vol_amt": ['vol_amt', 'sum', []],
                                           "vol_mf": ['signed_volamt', 'sum', []],
                                           "oi_mf": ['oi_amt', 'sum', []],
                                          }},
                   "s1": {"split_dict": [1500], 
                          "extra_fields": {"exec_price1": ['open', 'mean', [(1505, 1507)]],
                                           "exec_price2": ['open', 'mean', [(1510, 1512)]],
                                           "exec_price3": ['open', 'mean', [(1515, 1520)]],
                                           "exec_price4": ['open', 'mean', [(1520, 1525)]],
                                           "lr_mean": ['lr', 'mean', []],
                                           "lr_std": ['lr2', 'mean', []],
                                           "up_std": ['lr2_up', 'mean', []],
                                           "lr_skew": ['lr3', 'mean', []],
                                           "lr_kurt": ['lr4', 'mean', []],
                                           "min_cnt": ['lr', 'count', []],
                                           "lr_abs": ["lr_abs", 'mean', []],
                                           "vol_amt": ['vol_amt', 'sum', []],
                                           "vol_mf": ['signed_volamt', 'sum', []],
                                           "oi_mf": ['oi_amt', 'sum', []],
                                          }},
                   "s11": {"split_dict": [300], 
                          "extra_fields": {"exec_price1": ['open', 'mean', [(1505, 1507)]],
                                           "exec_price2": ['open', 'mean', [(1510, 1512)]],
                                           "exec_price3": ['open', 'mean', [(1515, 1520)]],
                                           "exec_price4": ['open', 'mean', [(1520, 1525)]],
                                           "lr_mean": ['lr', 'mean', []],
                                           "lr_std": ['lr2', 'mean', []],
                                           "up_std": ['lr2_up', 'mean', []],
                                           "lr_skew": ['lr3', 'mean', []],
                                           "lr_kurt": ['lr4', 'mean', []],
                                           "min_cnt": ['lr', 'count', []],
                                           "lr_abs": ["lr_abs", 'mean', []],
                                           "vol_amt": ['vol_amt', 'sum', []],
                                           "vol_mf": ['signed_volamt', 'sum', []],
                                           "oi_mf": ['oi_amt', 'sum', []],
                                          }},
                  }
                                                                 
for idx, asset in enumerate(sim_products):
    use_args = args
    if asset in eq_fut_mkts:
        use_args = eq_args
    elif asset in ['cu', 'al', 'zn', 'pb', 'sn']:
        use_args = base_args
    elif asset in ['ni']:
        use_args = base2_args
    elif asset in ['sc']:
        use_args = sc_args
    elif asset in ['v']:
        use_args = v_args
    elif asset in bond_fut_mkts:
        use_args = bond_args
    elif asset in precious_metal_mkts:
        use_args = precious_args
    use_args = copy.copy(use_args)
    use_args['start_date'] = max(sim_start_dict[asset], start_date)
    use_args['end_date'] = end_date
    for idy, nb in enumerate([1, 2]):
        use_args['n'] = nb
        print("loading mkt = %s, nb = %s, args = %s" % (asset, nb, use_args))
        df = misc.nearby(asset, **use_args)
        df = misc.cleanup_mindata(df, asset)
        if need_shift == 1:
            df['lr'] = np.log(df['close']-df['shift']) - np.log(df['close'].shift(1)-df['shift'])
        elif need_shift == 2:
            df['lr'] = np.log(df['close']) - np.log(df['close'].shift(1))
        else:
            df['lr'] = np.log(df['close']) - np.log(df['close'].shift(1))
        for pwr in [2, 3, 4]:
            df['lr%s' % str(pwr)] = df['lr']**pwr
        df['lr2_up'] = (df['lr']**2)*(df['lr']>0) 
        df['lr_abs'] = df['lr'].abs()
        df['vol_amt'] = df['close'] * df['volume']
        df['signed_volamt'] = ((df['lr'] > 0)*1.0-(df['lr'] <0)*1.0) * df['vol_amt']
        df['oi_amt'] = (df['openInterest'] - df['openInterest'].shift()).abs() * ((df['lr'] > 0)*1.0-(df['lr'] < 0)*1.0) * df['close']
        for sfreq in bar_split_setup:
            adf = df.reset_index().copy()
            for f in bar_split_setup[sfreq]['extra_fields']:
                field_setup = bar_split_setup[sfreq]['extra_fields'][f]
                if len(field_setup[2]) > 0:
                    adf[f] = np.nan
                    for min_range in field_setup[2]:
                        f_flag = (adf['min_id']>=min_range[0]) & (adf['min_id']<=min_range[1])
                        adf.loc[f_flag, f] = adf.loc[f_flag, field_setup[0]]
                else:
                    adf[f] = adf[field_setup[0]]
            adf = adf.set_index(['date', 'min_id'])
            field_funcs = {}
            for f in bar_split_setup[sfreq]['extra_fields']:
                field_setup = bar_split_setup[sfreq]['extra_fields'][f]
                field_funcs[f] = field_setup[1]
            xdf = minbar_agg(adf, bar_split_setup[sfreq]['split_dict'], \
                             index_col = ['date','min_id'], \
                             field_funcs = field_funcs)
            xdf['lr_kurt'] = xdf['lr_kurt']/(xdf['lr_std']**2)
            xdf['lr_skew'] = xdf['lr_skew']/(xdf['lr_std']**1.5)
            xdf['up_std'] = xdf['up_std']/xdf['lr_std']
            xdf['lr_std'] = xdf['lr_std'] * xdf['min_cnt']
            xdf['trd_str'] = xdf['lr_mean']/xdf['lr_abs']
            xdf['vol_mfratio'] = xdf['vol_mf']/xdf['vol_amt']
            filename = "%s_%s_%s_%s.csv" % (asset, use_args['n'], sfreq, end_date.strftime("%Y%m%d"))
            xdf = xdf.reset_index()
            xdf = xdf.drop_duplicates(subset=['date', 'split_idx'], keep='last')
            xdf.to_csv(file_folder + filename)


loading mkt = rb, nb = 1, args = {'roll_rule': '-30b', 'freq': 'm', 'need_shift': 1, 'dbtbl_prefix': '', 'start_date': datetime.date(2015, 1, 1), 'end_date': datetime.date(2021, 4, 7), 'n': 1}


C:\Anaconda3\lib\site-packages\pandas\core\indexing.py:1745: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  isetter(ilocs[0], value)


loading mkt = rb, nb = 2, args = {'roll_rule': '-30b', 'freq': 'm', 'need_shift': 1, 'dbtbl_prefix': '', 'start_date': datetime.date(2015, 1, 1), 'end_date': datetime.date(2021, 4, 7), 'n': 2}


C:\Anaconda3\lib\site-packages\pandas\core\indexing.py:1745: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  isetter(ilocs[0], value)


loading mkt = hc, nb = 1, args = {'roll_rule': '-30b', 'freq': 'm', 'need_shift': 1, 'dbtbl_prefix': '', 'start_date': datetime.date(2016, 4, 1), 'end_date': datetime.date(2021, 4, 7), 'n': 1}


...

loading mkt = jd, nb = 1, args = {'roll_rule': '-30b', 'freq': 'm', 'need_shift': 1, 'dbtbl_prefix': '', 'start_date': datetime.date(2015, 1, 1), 'end_date': datetime.date(2021, 4, 7), 'n': 1}


C:\Anaconda3\lib\site-packages\pandas\core\indexing.py:1745: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  isetter(ilocs[0], value)


loading mkt = jd, nb = 2, args = {'roll_rule': '-30b', 'freq': 'm', 'need_shift': 1, 'dbtbl_prefix': '', 'start_date': datetime.date(2015, 1, 1), 'end_date': datetime.date(2021, 4, 7), 'n': 2}


C:\Anaconda3\lib\site-packages\pandas\core\indexing.py:1745: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  isetter(ilocs[0], value)


loading mkt = AP, nb = 1, args = {'roll_rule': '-30b', 'freq': 'm', 'need_shift': 1, 'dbtbl_prefix': '', 'start_date': datetime.date(2018, 1, 1), 'end_date': datetime.date(2021, 4, 7), 'n': 1}
loading mkt = AP, nb = 2, args = {'roll_rule': '-30b', 'freq': 'm', 'need_shift': 1, 'dbtbl_prefix': '', 'start_date': datetime.date(2018, 1, 1), 'end_date': datetime.date(2021, 4, 7), 'n': 2}


In [ ]:
bar_split_setup = {"s2": {"split_dict": [300, 1500], 
                          "extra_fields": {"exec_price1": [(305, 307), (1505, 1507)],
                                           "exec_price2": [(310, 312), (1510, 1512)],
                                           "exec_price3": [(315, 320), (1515, 1520)],}},
                   "s21": {"split_dict": [1500, 1930], 
                          "extra_fields": {"exec_price1": [(1509, 1510), (1939, 1940)],}},
                   "s1": {"split_dict": [1500], 
                          "extra_fields": {"exec_price1": [(1505, 1507)],
                                           "exec_price2": [(1510, 1512)],
                                           "exec_price3": [(1515, 1520)],
                                           "exec_price4": [(1520, 1530)],}},
                   
                  }
file_folder = "C:\\dev\\data\\data_cache\\"
for idx, asset in enumerate(sim_products):
    use_args = args
    if asset in eq_fut_mkts:
        use_args = eq_args
    elif asset in ['cu', 'al', 'zn', 'pb', 'sn']:
        use_args = base_args
    elif asset in ['ni']:
        use_args = base2_args
    elif asset in ['sc']:
        use_args = sc_args
    elif asset in ['v']:
        use_args = v_args
    elif asset in bond_fut_mkts:
        use_args = bond_args
    elif asset in precious_metal_mkts:
        use_args = precious_args
    use_args = copy.copy(use_args)
    use_args['start_date'] = max(sim_start_dict[asset], start_date)
    use_args['end_date'] = end_date
    for idy, nb in enumerate([1, 2]):
        use_args['n'] = nb
        print("loading mkt = %s, nb = %s, args = %s" % (asset, nb, use_args))
        df = misc.nearby(asset, **use_args)
        df = misc.cleanup_mindata(df, asset)
        for sfreq in bar_split_setup:
            adf = df.reset_index().copy()
            for f in bar_split_setup[sfreq]['extra_fields']:
                adf[f] = np.nan
                for min_range in bar_split_setup[sfreq]['extra_fields'][f]:
                    f_flag = (adf['min_id']>=min_range[0]) & (adf['min_id']<=min_range[1])
                    adf.loc[f_flag, f] = adf.loc[f_flag, 'open']
            adf = adf.set_index(['date', 'min_id'])
            field_funcs = {}
            for f in bar_split_setup[sfreq]['extra_fields']:
                field_funcs[f] = 'mean'
            xdf = minbar_agg(adf, bar_split_setup[sfreq]['split_dict'], \
                             index_col = ['date','min_id'], \
                             field_funcs = field_funcs)
            filename = "%s_%s_%s_%s.csv" % (asset, use_args['n'], sfreq, end_date.strftime("%Y%m%d"))
            xdf = xdf.reset_index()
            xdf.to_csv(file_folder + filename)

In [25]:
print(df.tail(20))

                   rb-open  rb-high  rb-low  rb-close  rb-volume  \
date       min_id                                                  
2021-04-02 2040     5121.0   5121.0  5114.0    5114.0     4035.0   
           2041     5114.0   5119.0  5114.0    5117.0     2692.0   
           2042     5118.0   5120.0  5117.0    5119.0     1685.0   
           2043     5119.0   5120.0  5116.0    5117.0     2026.0   
           2044     5117.0   5119.0  5116.0    5119.0     1651.0   
           2045     5119.0   5124.0  5118.0    5121.0     5466.0   
           2046     5121.0   5124.0  5118.0    5123.0     3556.0   
           2047     5123.0   5126.0  5122.0    5123.0     4125.0   
           2048     5123.0   5124.0  5120.0    5121.0     2954.0   
           2049     5121.0   5125.0  5120.0    5124.0     2799.0   
           2050     5124.0   5125.0  5122.0    5123.0     2337.0   
           2051     5123.0   5141.0  5122.0    5134.0    11951.0   
           2052     5133.0   5139.0  5130.0    5

In [6]:
import pandas as pd
file_folder = "C:\\dev\\data\\data_cache\\"
asset = 'rb'
nb = 1
freq='s11'
end_date= '20210402'
filename = "%s_%s_%s_%s.csv" % (asset, nb, freq, end_date)
df = pd.read_csv(file_folder + filename, index_col=0).set_index(['date', 'split_idx'])
df = df.drop(['cum_idx'], axis=1)
print(df.tail(10))

                      min_id             datetime  instID  exch    open  \
date       split_idx                                                      
2021-03-22 0            2059  2021-03-22 21:00:00  rb2105  SHFE  4763.0   
2021-03-23 0            2059  2021-03-23 21:00:00  rb2105  SHFE  4762.0   
2021-03-24 0            2059  2021-03-24 21:00:00  rb2105  SHFE  4703.0   
2021-03-25 0            2059  2021-03-25 21:01:00  rb2105  SHFE  4810.0   
2021-03-26 0            2059  2021-03-26 21:01:00  rb2105  SHFE  4789.0   
2021-03-29 0            2059  2021-03-29 21:00:00  rb2105  SHFE  4940.0   
2021-03-30 0            2059  2021-03-30 21:01:00  rb2105  SHFE  4973.0   
2021-03-31 0            2059  2021-03-31 21:01:00  rb2105  SHFE  4956.0   
2021-04-01 0            2059  2021-04-01 21:01:00  rb2105  SHFE  4938.0   
2021-04-02 0            2059  2021-04-02 21:01:00  rb2105  SHFE  5039.0   

                        high     low   close     volume  openInterest  ...  \
date       split_idx 